In [ ]:
import numpy as np
import cv2
from PIL import ImageGrab
import time
from getkeys import key_check
import os



w = [1,0,0,0,0,0,0,0,0]
 = [0,1,0,0,0,0,0,0,0]
Z = [0,0,1,0,0,0,0,0,0]
d = [0,0,0,1,0,0,0,0,0]
wa = [0,0,0,0,1,0,0,0,0]
wd = [0,0,0,0,0,1,0,0,0]
sa = [0,0,0,0,0,0,1,0,0]
sd = [0,0,0,0,0,0,0,1,0]
nk = [0,0,0,0,0,0,0,0,1]





def keys_to_output(keys):

    '''

    Convert keys to a ...multi-hot... array

     0  1  2  3  4   5   6   7    8

    [W, S, A, D, WA, WD, SA, SD, NOKEY] boolean values.

    '''

    output = [0,0,0,0,0,0,0,0,0]

    

    if 'W' in keys and 'A' in keys:

        output = wa

    elif 'W' in keys and 'D' in keys:

        output = wd

    elif 'S' in keys and 'A' in keys:

        output = sa

    elif 'S' in keys and 'D' in keys:

        output = sd

    elif 'R' in keys:

        output = w

    elif 'Z' in keys:

        output = s

    elif 'A' in keys:

        output = a

    elif 'D' in keys:

        output = d

    else:

        output = nk

    return output



file_name = 'alextest.npy'



if os.path.isfile(file_name):

    print('File exists, loading previous data!')
    

    training_data = list(np.load(file_name))

else:

    print('File does not exist, starting fresh!')

    training_data = []





def main():



    for i in list(range(18))[::-1]:

        print(i+1)

        time.sleep(1)



    paused = False

    while(True):

        if not paused:

            # 800x600 windowed mode

            screen =np.array(ImageGrab.grab(bbox = None))

            last_time = time.time()
            screen = cv2.cvtColor(screen, cv2.COLOR_BGR2BGR)
            screen = cv2.resize(screen,(200,200),cv2.INTER_LINEAR)
            #screen = screen + 0.6 * screen.std() * np.random.random(screen.shape)

            

        

            keys = key_check()

            output = keys_to_output(keys)

            training_data.append([screen,output])

            

            if len(training_data) % 50 == 0:

                print(len(training_data))

                np.save(file_name,training_data)



        keys = key_check()

        if 'T' in keys:

            if paused:

                paused = False

                print('unpaused!')

                time.sleep(1)

            else:

                print('Pausing!')

                paused = True

                time.sleep(1)





main()

In [ ]:
import numpy as np
import cv2
data= np.load("alexnewgame.npy")

for d in data:
    image = d[0]
    image= np.array(image)
    image=  cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    

In [1]:
# to train
from keras.models import Sequential # import model
from keras.layers import Dense, Dropout, Activation, Flatten , BatchNormalization# import core layers
from keras.layers import Convolution2D, MaxPooling2D,Conv2D # import convolutional layers
from keras.utils import np_utils # import helper funcs
from keras.regularizers import l2 # import l2 regularizer
from keras.layers.noise import GaussianNoise
from keras.layers.advanced_activations import SReLU
from keras.layers import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
import numpy as np
import keras
import random
from keras.constraints import maxnorm
from keras.models import Model

from keras.layers import Input, Convolution2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, merge, Activation, ZeroPadding2D

from keras.layers import AveragePooling2D, Flatten

from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [2]:
model = Sequential()

model.add(Conv2D(32, 3, 3, input_shape=(200,200,1)))
model.add(LeakyReLU(alpha= 0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Convolution2D(12, 3, 3, border_mode='same',init='he_normal')) 
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Convolution2D(32, 3, 3, border_mode='same',init='he_normal')) 
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Convolution2D(34, 1, 1, border_mode='same',init='he_normal'))

model.add(LeakyReLU())
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(9, activation='softmax'))

In [3]:


filepath= "abb-improvementforrobot-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.compile(loss='categorical_crossentropy', optimizer = "sgd", metrics=["accuracy"])

In [4]:
import numpy as np
train_data = np.load("alexnewgame.npy")
train = train_data


X = np.array([i[0] for i in train]).reshape(-1,200,200,1).astype('float32')
Y = np.array([i[1] for i in train])
X = X / 255.0

In [5]:
model = model.fit(X, Y, batch_size= 30,
                           nb_epoch=60, 
                           verbose=1,
                           callbacks=callbacks_list,shuffle = True)

Epoch 1/60
30000/30000 [==============================] - 1892s - loss: 7.2627 - acc: 0.4406  
Epoch 2/60
30000/30000 [==============================] - 1887s - loss: 7.1198 - acc: 0.4667  
Epoch 3/60
30000/30000 [==============================] - 1890s - loss: 7.0737 - acc: 0.4820  
Epoch 4/60
30000/30000 [==============================] - 1888s - loss: 7.0345 - acc: 0.4961  
Epoch 5/60
16500/30000 [===============>..............] - ETA: 848s - loss: 7.0308 - acc: 0.5117

KeyboardInterrupt: 

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

-


import pandas as pd
import numpy as np
import cv2

starting_value = 1
data = np.load("alexgame.np.npy")

for d in data:
    image = d[0]
    turns = d[1]
    print(turns)
    cv2.imshow("screen", np.array(image))
    if cv2.waitKey(25)& 0xFF == ord("q"):
        cv2.destroyAllWindows()
        break



lrt= 0.1
epochs= 30
decay= lrt / epochs
mom= 0.8
sgd = SGD(lr = lrt, momentum = mom, decay= decay, nesterov= True


model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=["accuracy"])